In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import pandas as pd

from depsurf import FileLogger, check_diff, Kind, LinuxImage, GenericChange


def diff_btf(img1: LinuxImage, img2: LinuxImage, path):
    results = {}
    for key, kind in [
        ("function", Kind.FUNC),
        ("struct", Kind.STRUCT),
        # ("union", Kind.UNION),
        # ("enum", Kind.ENUM),
        ("tp_event", Kind.STRUCT),
        ("tp_func", Kind.FUNC),
        ("lsm", Kind.FUNC),
    ]:
        if key == "lsm":
            dict1 = img1.lsm_hooks
            dict2 = img2.lsm_hooks
        elif key == "tp_func":
            dict1 = img1.tracepoints.funcs
            dict2 = img2.tracepoints.funcs
        elif key == "tp_event":
            dict1 = img1.tracepoints.events
            dict2 = img2.tracepoints.events
        else:
            dict1 = img1.btf.get_all_kind(kind)
            dict2 = img2.btf.get_all_kind(kind)

        output_path = path / f"{key}.log"
        with FileLogger(output_path):
            diff_result = check_diff(dict1, dict2)
        results[(key, "Old")] = len(dict1)
        # results[(key, "New")] = len(dict2)
        results[(key, GenericChange.ADD)] = len(diff_result.added)
        results[(key, GenericChange.REMOVE)] = len(diff_result.removed)
        # results[(key, "Common")] = len(diff_result.common)
        # results[(key, "Changed")] = len(diff_result.changed)
        for k, v in diff_result.reasons.items():
            results[(key, k)] = v

    df = pd.DataFrame(
        results.values(),
        index=pd.MultiIndex.from_tuples(results.keys()),
        columns=["Count"],
    )
    df.to_string(path / "summary.txt")
    df.to_pickle(path / "summary.pkl")
    return df


# diff_btf(LinuxImage.from_str("5.4.0-26-generic-amd64"), LinuxImage.from_str("5.8.0-25-generic-amd64"), "dist", "test")

In [4]:
from itertools import pairwise

from depsurf import OUTPUT_PATH, BuildVersion

results = {}


def diff_images(group, versions):
    print(f"Diffing {group}: {versions})")
    for v1, v2 in pairwise(versions):
        path = OUTPUT_PATH / group / f"{v1}_{v2}"
        print(f"Comparing {v1} and {v2} to {path}")
        df = diff_btf(v1.img, v2.img, path)
        results[(group, (v1.name, v2.name))] = df


diff_images("lts", BuildVersion.filter(flavor="generic", arch="amd64", lts=True))
diff_images("all", BuildVersion.filter(flavor="generic", arch="amd64"))
diff_images(
    "rev",
    BuildVersion.filter(
        flavor="generic", arch="amd64", revision="all", version="5.4.0"
    ),
)

[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/4.4.0-21-generic-amd64.pkl


Diffing lts: [4.4.0-21-generic-amd64, 4.15.0-20-generic-amd64, 5.4.0-26-generic-amd64, 5.15.0-25-generic-amd64, 6.8.0-22-generic-amd64])
Comparing 4.4.0-21-generic-amd64 and 4.15.0-20-generic-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/lts/4.4.0-21-generic-amd64_4.15.0-20-generic-amd64


[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/4.15.0-20-generic-amd64.pkl
[   symtab.py:33 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.4.0-21-generic-amd64.pkl
[tracepoint.py:59 ] WARNING: Could not find function for xen_mmu_flush_tlb_all
[   symtab.py:33 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/4.15.0-20-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-26-generic-amd64.pkl


Comparing 4.15.0-20-generic-amd64 and 5.4.0-26-generic-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/lts/4.15.0-20-generic-amd64_5.4.0-26-generic-amd64


[   symtab.py:33 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-26-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.15.0-25-generic-amd64.pkl


...

[   symtab.py:33 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-100-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-125-generic-amd64.pkl


Comparing 5.4.0-100-generic-amd64 and 5.4.0-125-generic-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/rev/5.4.0-100-generic-amd64_5.4.0-125-generic-amd64


[   symtab.py:33 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-125-generic-amd64.pkl
[      btf.py:16 ] INFO: Loading BTF from /Users/szhong/Downloads/bpf-study/btf/data/btf_norm/5.4.0-175-generic-amd64.pkl


Comparing 5.4.0-125-generic-amd64 and 5.4.0-175-generic-amd64 to /Users/szhong/Downloads/bpf-study/btf/output/rev/5.4.0-125-generic-amd64_5.4.0-175-generic-amd64


[   symtab.py:33 ] INFO: Loading symbol table from /Users/szhong/Downloads/bpf-study/btf/data/symtab/5.4.0-175-generic-amd64.pkl


In [ ]:
df = pd.concat(results, axis=1)
df.columns = df.columns.droplevel(2)
df = df.loc[(df != 0).any(axis=1)]
df.to_pickle(OUTPUT_PATH / "diff.pkl")

In [5]:
import pandas as pd
from depsurf import setup_pandas, OUTPUT_PATH

setup_pandas()

pd.read_pickle(OUTPUT_PATH / "diff.pkl")

lts                                                                                                                                                                                                    all                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               \
                             (4.4.0-21-generic-amd64, 4.15.0-20-generic-amd64) (4.15.0-20-generic-amd64, 5.4.0-26-generic-amd64) (5.4.0-26-generic-amd64, 5.15.0-25-generic-amd64) (5.15.0-25-generic-amd64, 6.8.0-22-generic-amd64) (4.4.0-21-generic-amd64, 4.8.0-22-generic-amd64) (4.8.0-22-generic-amd64, 4.10.0-19-generic-amd64) (4.10.0-19-generic-amd64, 4.13.0-16-generic-amd64) (4.13.0-16-generic-amd64, 4.15.0-20-generic-amd64) (4.15.0-20-generic-amd64, 4.18.0-10-generic-amd64) (4.18.0-10-generic-amd64, 5.0.0-13-generic-amd64) (5.0.0-13-generic-amd64, 5.3.0-18-generic-amd64) (5.3.0-18-generic-amd64, 5.4.0-26-generic-amd64) (5.4.0-26-generic-amd64, 5.8.0-25-generic-amd64) (5.8.0-25-generic-amd64, 5.11.0-16-generic-amd64) (5.11.0-16-generic-amd64, 5.13.0-19-generic-amd64) (5.13.0-19-generic-amd64, 5.15.0-25-generic-amd64) (5.15.0-25-generic-amd64, 5.19.0-21-generic-amd64) (5.19.0-21-generic-amd64, 6.2.0-20-generic-amd64) (6.2.0-20-generic-amd64, 6.5.0-9-generic-amd64)   
function Old                                                             36309                                             42299                                             48018                                             53956                                            36309                                             38211                                              39148                                              40741                                              42299                                             44488                                            45277                                            46992                                            48018                                             51936                                              54302                                              53193                                              53956                                             55843                                           56978   
         Added                                                            8600                                              8929                                             10771                                             12139                                             2849                                              1574                                               2662                                               2118                                               3635                                              1980                                             2516                                             1382                                             6785                                              3518                                               1469                                               1753                                               3525                                              2622                                            3196   
         Removed                                                          2610                                              3210                   